# Check for Understanding — Autograded (PyTorch)
Run each cell. **Do not delete the asserts.**

**Passing condition:** all asserts pass.

Tip: If an assert fails, read its message, fix your code, and rerun the cell.


In [17]:
# Setup
import torch
import torch.nn as nn

torch.manual_seed(42)

def _is_close(a, b, tol=1e-5):
    return torch.allclose(a, b, atol=tol, rtol=0)

print("PyTorch version:", torch.__version__)


PyTorch version: 2.10.0


## Part 1 — Tensors & Representations

In [18]:
# Exercise 1: Tensor basics
# TODO:
# 1) Create a 2x3 tensor of random values called X
# 2) Print X, X.shape, X.dtype
# 3) Compute the mean of all elements and store it in x_mean (a 0-d tensor)

X = torch.randn(2, 3)  # YOUR CODE HERE
x_mean = X.mean()  # YOUR CODE HERE

# Print statements (uncomment after implementing)
print("X=\n", X)
print("shape:", X.shape)
print("dtype:", X.dtype)
print("mean:", x_mean)

# --- autograder asserts (do not delete) ---
assert isinstance(X, torch.Tensor), "X must be a torch.Tensor"
assert X.shape == (2, 3), f"X must have shape (2,3), got {tuple(X.shape)}"
assert X.dtype in (torch.float32, torch.float64), f"X should be float32/float64, got {X.dtype}"
assert isinstance(x_mean, torch.Tensor) and x_mean.shape == (), "x_mean must be a scalar (0-d) tensor"
assert _is_close(x_mean, X.sum() / X.numel()), "x_mean should equal X.sum()/X.numel()"


X=
 tensor([[ 0.3367,  0.1288,  0.2345],
        [ 0.2303, -1.1229, -0.1863]])
shape: torch.Size([2, 3])
dtype: torch.float32
mean: tensor(-0.0631)


In [19]:
# Exercise 2: Manual vector operations
# TODO:
# 1) Create v1 and v2 as 1-D tensors of length 5
# 2) Compute element-wise sum: v_sum
# 3) Compute dot product: v_dot (scalar tensor)

v1 = torch.randn(5)  # YOUR CODE HERE
v2 = torch.randn(5)  # YOUR CODE HERE

v_sum = v1 + v2  # YOUR CODE HERE
v_dot = torch.dot(v1, v2)  # YOUR CODE HERE

# Print statements (uncomment after implementing)
print("v1:", v1)
print("v2:", v2)
print("v_sum:", v_sum)
print("v_dot:", v_dot)

# --- autograder asserts (do not delete) ---
assert v1.shape == (5,) and v2.shape == (5,), "v1 and v2 must both be shape (5,)"
assert v_sum.shape == (5,), "v_sum must be a length-5 vector"
assert v_dot.shape == (), "v_dot must be a scalar (0-d) tensor"
manual_dot = (v1 * v2).sum()
assert _is_close(v_dot, manual_dot), "v_dot must equal (v1*v2).sum()"


v1: tensor([ 2.2082, -0.6380,  0.4617,  0.2674,  0.5349])
v2: tensor([ 0.8094,  1.1103, -1.6898, -0.9890,  0.9580])
v_sum: tensor([ 3.0176,  0.4723, -1.2281, -0.7216,  1.4929])
v_dot: tensor(0.5468)


## Part 2 — Embeddings

In [20]:
# Exercise 3: Simple embedding lookup
# TODO:
# 1) Create an nn.Embedding called emb with vocab_size=10 and emb_dim=4
# 2) Create token_ids as a LongTensor of shape (3,) with values in [0, 9]
# 3) Lookup embeddings: E = emb(token_ids)
# 4) Print E and E.shape

vocab_size, emb_dim = 10, 4
emb = nn.Embedding(vocab_size, emb_dim)  # YOUR CODE HERE

token_ids = torch.tensor([1, 5, 9], dtype=torch.long)  # YOUR CODE HERE
E = emb(token_ids)  # YOUR CODE HERE

# Print statements (uncomment after implementing)
print("token_ids:", token_ids)
print("E=\n", E)
print("E.shape:", E.shape)

# --- autograder asserts (do not delete) ---
assert isinstance(emb, nn.Embedding), "emb must be an nn.Embedding"
assert token_ids.dtype == torch.long, "token_ids must be torch.long"
assert token_ids.shape == (3,), f"token_ids must be shape (3,), got {tuple(token_ids.shape)}"
assert E.shape == (3, 4), f"E must have shape (3,4), got {tuple(E.shape)}"
assert E.requires_grad, "Embedding output should require gradients by default"


token_ids: tensor([1, 5, 9])
E=
 tensor([[ 0.3189, -0.4245,  0.3057, -0.7746],
        [-0.4880,  1.1914, -0.8140, -0.7360],
        [-0.5644,  1.0563, -1.4692,  1.4332]], grad_fn=<EmbeddingBackward0>)
E.shape: torch.Size([3, 4])


In [21]:
# Exercise 4: From embeddings to a prediction
# NOTE: This exercise depends on Exercise 3 — complete that first.
# TODO:
# 1) Compute mean embedding across tokens: mean_E of shape (4,)
# 2) Create a Linear layer (4 -> 1) called head
# 3) Produce y_pred as shape (1,) or scalar

mean_E = E.mean(dim=0)  # YOUR CODE HERE
head = nn.Linear(4, 1)  # YOUR CODE HERE

y_pred = head(mean_E)  # YOUR CODE HERE

# Print statements (uncomment after implementing)
print("mean_E.shape:", mean_E.shape)
print("y_pred:", y_pred, "shape:", y_pred.shape)

# --- autograder asserts (do not delete) ---
assert mean_E.shape == (4,), f"mean_E must be shape (4,), got {tuple(mean_E.shape)}"
assert isinstance(head, nn.Linear) and head.in_features == 4 and head.out_features == 1, "head must be Linear(4->1)"
assert y_pred.numel() == 1, "y_pred must have exactly 1 element"
assert y_pred.requires_grad, "y_pred should require gradients"


mean_E.shape: torch.Size([4])
y_pred: tensor([0.3860], grad_fn=<ViewBackward0>) shape: torch.Size([1])


## Part 3 — Build a Tiny Network

In [22]:
# Exercise 5: Define a simple feed-forward network
# Requirements:
# - input_dim = 6
# - hidden_dim = 8
# - output_dim = 1
# - 1 hidden layer + ReLU
# Implement SimpleNet so forward(x) returns shape (batch, 1)

class SimpleNet(nn.Module):
    def __init__(self, input_dim=6, hidden_dim=8, output_dim=1):
        super().__init__()
        # YOUR CODE HERE — define layers (fc1, fc2, activation)
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.activation = nn.ReLU()
        self.fc2 = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        # YOUR CODE HERE — implement forward pass
        x = self.fc1(x)
        x = self.activation(x)
        x = self.fc2(x)
        return x

model = SimpleNet()
print(model)

# --- autograder asserts (do not delete) ---
assert isinstance(model, nn.Module), "model must be an nn.Module"
params = dict(model.named_parameters())
assert "fc1.weight" in params and "fc2.weight" in params, "Model must have two Linear layers (fc1, fc2)"


SimpleNet(
  (fc1): Linear(in_features=6, out_features=8, bias=True)
  (activation): ReLU()
  (fc2): Linear(in_features=8, out_features=1, bias=True)
)


In [23]:
# Exercise 6: Forward pass with dummy data
# NOTE: This exercise depends on Exercise 5 — complete that first.
# TODO:
# 1) Create dummy input x of shape (4, 6)
# 2) Run out = model(x)
# 3) Print out and out.shape

x = torch.randn(4, 6)  # YOUR CODE HERE
out = model(x)  # YOUR CODE HERE

# Print statements (uncomment after implementing)
print("out=\n", out)
print("out.shape:", out.shape)

# --- autograder asserts (do not delete) ---
assert x.shape == (4, 6), f"x must be shape (4,6), got {tuple(x.shape)}"
assert out.shape == (4, 1), f"out must be shape (4,1), got {tuple(out.shape)}"


out=
 tensor([[0.1829],
        [0.4387],
        [0.2505],
        [0.4122]], grad_fn=<AddmmBackward0>)
out.shape: torch.Size([4, 1])


## Part 4 — One Training Step

In [24]:
# Exercise 7: One training step
# NOTE: This exercise depends on Exercise 5 — complete that first.
# TODO:
# 1) Create inputs x_train (batch=8, input_dim=6) and targets y_train (shape (8,1))
# 2) Define loss_fn = MSELoss and opt = SGD(model.parameters(), lr=0.1)
# 3) Perform exactly one update step and print loss_before and loss_after

torch.manual_seed(123)  # deterministic for this part

# Create training data (provided for you)
x_train = torch.randn(8, 6)
true_w = torch.tensor([[0.5], [-1.0], [0.3], [0.0], [1.2], [-0.7]])
y_train = x_train @ true_w + 0.01 * torch.randn(8, 1)

loss_fn = nn.MSELoss()  # YOUR CODE HERE
opt = torch.optim.SGD(model.parameters(), lr=0.1)  # YOUR CODE HERE

# YOUR CODE HERE — compute loss_before (forward pass + loss)
loss_before = loss_fn(model(x_train), y_train)

# YOUR CODE HERE — perform backward pass and optimizer step
loss_before.backward()
opt.step()
opt.zero_grad()

# YOUR CODE HERE — compute loss_after (forward pass + loss)
loss_after = loss_fn(model(x_train), y_train)

# Print statements (uncomment after implementing)
print("loss_before:", float(loss_before))
print("loss_after :", float(loss_after))

# --- autograder asserts (do not delete) ---
assert loss_before.shape == (), "loss_before must be a scalar tensor"
assert loss_after.shape == (), "loss_after must be a scalar tensor"
assert float(loss_after) < float(loss_before), "loss_after should be < loss_before after one SGD step"


loss_before: 1.5991227626800537
loss_after : 1.193355917930603


## Optional Stretch (No grade)
If you finish early:
1. Add a second training step and show loss keeps decreasing.
2. Change activation to Tanh and compare loss curves.


In [25]:
# Stretch 1: Second training step — show loss keeps decreasing
# We already have model, loss_fn, opt, x_train, y_train from Exercise 7

loss_step2_before = loss_fn(model(x_train), y_train)

# Second SGD step
opt.zero_grad()
loss_step2_before.backward()
opt.step()

loss_step2_after = loss_fn(model(x_train), y_train)

print("loss after 1st step:", float(loss_after))
print("loss after 2nd step:", float(loss_step2_after))
assert float(loss_step2_after) < float(loss_after), "Loss should keep decreasing!"
print(" Loss kept decreasing after the 2nd step!")


loss after 1st step: 1.193355917930603
loss after 2nd step: 1.0456650257110596
 Loss kept decreasing after the 2nd step!


In [ ]:
# Stretch 2: Change activation to Tanh and compare loss curves
# Build a Tanh version of SimpleNet and train both for 20 steps

class SimpleNetTanh(nn.Module):
    def __init__(self, input_dim=6, hidden_dim=8, output_dim=1):
        super().__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.activation = nn.Tanh()
        self.fc2 = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        x = self.fc1(x)
        x = self.activation(x)
        x = self.fc2(x)
        return x

# Train both models from scratch for 20 steps and record losses
torch.manual_seed(42)
model_relu = SimpleNet()
model_tanh = SimpleNetTanh()

# Use same training data
torch.manual_seed(123)
x_tr = torch.randn(8, 6)
true_w = torch.tensor([[0.5], [-1.0], [0.3], [0.0], [1.2], [-0.7]])
y_tr = x_tr @ true_w + 0.01 * torch.randn(8, 1)

loss_fn = nn.MSELoss()
opt_relu = torch.optim.SGD(model_relu.parameters(), lr=0.01)
opt_tanh = torch.optim.SGD(model_tanh.parameters(), lr=0.01)

losses_relu = []
losses_tanh = []

for step in range(20):
    # ReLU model
    opt_relu.zero_grad()
    l_relu = loss_fn(model_relu(x_tr), y_tr)
    losses_relu.append(float(l_relu))
    l_relu.backward()
    opt_relu.step()

    # Tanh model
    opt_tanh.zero_grad()
    l_tanh = loss_fn(model_tanh(x_tr), y_tr)
    losses_tanh.append(float(l_tanh))
    l_tanh.backward()
    opt_tanh.step()

# Print comparison
print("Step | ReLU Loss  | Tanh Loss")
print("-----|------------|----------")
for i in range(20):
    print(f"  {i+1:2d} | {losses_relu[i]:.6f}   | {losses_tanh[i]:.6f}")

print(f"\nFinal ReLU loss:  {losses_relu[-1]:.6f}")
print(f"Final Tanh loss:  {losses_tanh[-1]:.6f}")
print("Comparison complete!")


Step | ReLU Loss  | Tanh Loss
-----|------------|----------
   1 | 1.353543   | 1.226476
   2 | 1.331813   | 1.193727
   3 | 1.311439   | 1.162124
   4 | 1.292291   | 1.131598
   5 | 1.274254   | 1.102089
   6 | 1.257225   | 1.073542
   7 | 1.241109   | 1.045911
   8 | 1.225821   | 1.019150
   9 | 1.211285   | 0.993223
  10 | 1.197986   | 0.968093
  11 | 1.186279   | 0.943731
  12 | 1.175037   | 0.920108
  13 | 1.164224   | 0.897199
  14 | 1.153804   | 0.874982
  15 | 1.143792   | 0.853435
  16 | 1.134138   | 0.832539
  17 | 1.124800   | 0.812276
  18 | 1.115753   | 0.792631
  19 | 1.106974   | 0.773587
  20 | 1.098441   | 0.755129

Final ReLU loss:  1.098441
Final Tanh loss:  0.755129
✅ Comparison complete!
